## Лабораторная работа 3. Работа с веб-камерой

### Цель: Отработать навыки работы с веб-камерой с использованием OpenCV.

Задачи
1. Считать видеопоток с веб-камеры, реализовать отображение видеопотока в блокноте.
2. Детектировать лица на кадрах видеопотока с использованием метода cv2.CascadeClassifier.
3. Экстрактировать признаки из изображения детектированного лица.
4. Создать набор данных (признаков) своего лица.
5. Реализовать метод сравнения признаков лица, детектируемого в потоке, с набором признаков из п.4. В случае нахождения "похожего" лица в наборе данных признать лицо в видеопотоке идентифицированным (eg. "Павел Якимов"), иначе: "Unknown person".
6. Вывести результат идентификации на экран

In [1]:
import cv2
import os
import numpy as np

from PIL import Image

from IPython.display import display, Image, clear_output

%pylab inline

#from matplotlib import pyplot as plt

Populating the interactive namespace from numpy and matplotlib


### Считываем видеопоток с веб-камеры

In [2]:
# параметр cv2.CAP_DSHOW исправляет "перевернутое изображение"

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)


try:
    while True:
        ret, frame = cap.read()
               
        if not ret:
            cap.release()
            print('Released Video Resource') 
            break
        
        if cv2.waitKey(10) == 27: # Клавиша Esc
            break
        
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #убрали оси
        axis('off')
        # название вывели
        title('Input Stream')
        # выводим на экран
        imshow(frame)
        show()
        clear_output(wait = True)
    
except KeyboardInterrupt:
    cap.release()
    print('Released Video Resource')
 

Released Video Resource


### Определение лица

In [3]:
def find_face(image):
    
    #face_cascade = cv2.CascadeClassifier(haarcascade_frontalface_default.xml")
    
    # не cчитал загруженный каскад, но сработал такой вот фрагмент кода:
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
        
    # конвертируем в оттенки серого
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # faces = face_cascade.detectMultiScale(gray, scaleFactor, minNeighbors, minSize)
    faces = face_cascade.detectMultiScale(gray, 1.2, 15)
    
    # рисуем прямоугольник вокруг лица
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
    #print('Faces found: ', len(faces))
    return image   

In [4]:
video = cv2.VideoCapture(0, cv2.CAP_DSHOW)

display_handle=display(None, display_id=True)

try:
    while True:
        ret, frame = video.read()
        if not ret:
            video.release()
            print ("Released Video Resource")
            break
        
        frame = find_face(frame)
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))

except KeyboardInterrupt:
    pass

finally:
    video.release()
    display_handle.update(None)

None

### Сбор данных, создание набора данных

In [5]:
# mkdir data_face
# mkdir data_eyes
# mkdir data_smile


In [11]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height

face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
smile_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")

# указать имя пользователя, которое буде в названии файлов
face_id = input('\n Введите имя  ==>  ')

print("\n Пожалуйста, смотрите в камеру")


count1 = 0
count2 = 0
count3 = 0

while(True):
    ret, img = cam.read()
    #img = cv2.flip(img, -1) # flip video image vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    eyes = eye_detector.detectMultiScale(gray, 1.3, 5)
    smiles = smile_detector.detectMultiScale(gray, 1.3, 5)
    

    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count1 += 1
        # сохраняем изображения в папку 
        cv2.imwrite("data_face/" + str(face_id) + '.' + str(count1) + ".jpg", gray[y:y+h,x:x+w])
        cv2.imshow('image', img)

        
    for (x,y,w,h) in eyes:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count2 += 1
        cv2.imwrite("data_eyes/" + str(face_id) + '.' + str(count2) + ".jpg", gray[y:y+h,x:x+w])
        cv2.imshow('image', img)    
        
        
#    for (x,y,w,h) in smiles:
#        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
#        count3 += 1
#        cv2.imwrite("data_smile/" + str(face_id) + '.' + str(count3) + ".jpg", gray[y:y+h,x:x+w])
#        cv2.imshow('image', img)
        
        
        
    # 'ESC' чтобы остановить видео
    k = cv2.waitKey(100) & 0xff 
    if k == 27:
        break
    
    # Делаем 50 фоток
    elif count1 >= 50: 
         break

print("\n Набор данных для пользователя", face_id, "создан")

cam.release()
cv2.destroyAllWindows()


 Введите имя  ==>  Julia

 Пожалуйста, смотрите в камеру

 Набор данных для пользователя Julia создан


#### обучаем

In [13]:
# mkdir train

In [14]:
#  будет принимать все фотографии по указанному пути, возвращая 2 массива: «Идентификаторы(lds)» и «Лица(Faces)»
from PIL import Image
def getImg(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)] 
    #print(imagePaths)
    faceSamples=[]
    ids = []
    
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    
    return faceSamples,ids

In [21]:
# путь к файлам
#path_ = "data_face"
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# обучаем
faces, ids = getImg("data_face")
recognizer.train(faces, np.array(ids))
# сохраняем результат обучения в файл
recognizer.write('train/trainer_face.yml') 


#recognizer = cv2.face.LBPHFaceRecognizer_create()
#detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
#faces, ids = getImg("data_eyes")
#recognizer.train(faces, np.array(ids))
#recognizer.write('train/trainer_eyes.yml')


#recognizer = cv2.face.LBPHFaceRecognizer_create()
#detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")
#faces, ids = getImg("data_smile")
#recognizer.train(faces, np.array(ids))
#recognizer.write('train/trainer_smile.yml')




#### Распознаем


только по лицу (но можно также и по другим признакам)

In [22]:
recognizer_face = cv2.face.LBPHFaceRecognizer_create()
recognizer_face.read('train/trainer_face.yml')
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

#recognizer_eye = cv2.face.LBPHFaceRecognizer_create()
#recognizer_eye.read('train/trainer_eyes.yml')
#eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


font = cv2.FONT_HERSHEY_SIMPLEX


In [27]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

try:
    while True:
        ret, img = cam.read()
               
        if not ret:
            cam.release()
            print('Released Video Resource') 
            break
        
        if cv2.waitKey(10) == 27: # Клавиша Esc
            break
        
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
        faces = faceCascade.detectMultiScale(gray, 1.2, 5)
        
        for(x,y,w,h) in faces:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

            if (confidence < 70):
                
                id = "Julia"
                 
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
        
        # вероятность в процентах
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)
        
              
        
        #убрали оси
        axis('off')
        
        # выводим на экран
        imshow(img)
        show()
        clear_output(wait = True)
    
except KeyboardInterrupt:
    cam.release()
    print('Released Video Resource')
 

Released Video Resource


Результат, конечно, будет зависеть от:
- объема набора данных (у меня 50 снимков)
- точности (у меня это confidence < 70)
- от количества признаков (глаза, улыбка и т.д.). 
